In [1]:
import pymssql
import pandas as pd
import re

In [2]:
SQL_HOST = '*****'
SQL_USER = '*****'
SQL_PWD = '*****'
SQL_DB = '*****'

sql_connect = pymssql.connect(
        server=SQL_HOST,
        user=SQL_USER,
        password=SQL_PWD,
        database=SQL_DB,
        as_dict=True,
        charset="UTF-8",
        timeout=200
    )

In [3]:
cursor_sql = sql_connect.cursor()
cursor_sql.execute("****")
data = cursor_sql.fetchall()

In [4]:
data_df = pd.DataFrame(data)
data_df.head(3)

,id,Title,TitleEN,TitleSearch,TitleVN,TopTitle,State,Status,Image,Actors,...,AppID,Folder,ListOnfolder,EndTimeMovie,MPA,Producers,Parental,Is4K,Image_New,Image_4K
0,100020536,Amazon: Dòng Sông Của Mặt Trời (The Amazon Riv...,The Amazon River Of The Sun,Amazon: Dong Song Cua Mat Troi (The Amazon Riv...,Amazon: Dòng Sông Của Mặt Trời,Tài liệu,5,1,http://fboximages.fptplay.net.vn/VOD/Documenta...,Narrated,...,21,16,16,2100-01-01,PG,Razor Digital Entertainment,0,0,http://fboximages.fptplay.net.vn/VOD/Documenta...,http://fboximages.fptplay.net.vn/VOD/Documenta...
1,100047007,Khai Quật Tổ Quỷ (Digging Up The Marrow),Digging Up The Marrow,Khai Quat To Quy (Digging Up The Marrow),Khai Quật Tổ Quỷ,Tài liệu,5,1,http://fboximages.fptplay.net.vn/VOD/Documenta...,"Ray Wise, Adam Green, Will Barratt",...,21,16,"16,300",2100-01-01,R,ArieScope Pictures,16,0,http://fboximages.fptplay.net.vn/VOD/Documenta...,http://fboximages.fptplay.net.vn/VOD/Documenta...
2,100028086,Sóng Thần Nhật Bản Đã Xảy Ra Như Thế Nào? (Jap...,Japans Tsunami - How It Happened,Song Than Nhat Ban Da Xay Ra Nhu The Nao? (Jap...,Sóng Thần Nhật Bản Đã Xảy Ra Như Thế Nào?,Tài liệu,5,1,http://fboximages.fptplay.net.vn/VOD/Documenta...,"Mark Strong, Roger Bilham, Simon Boxall",...,21,16,16,2100-01-01,PG,Pioneer Productions,0,0,http://fboximages.fptplay.net.vn/VOD/Documenta...,http://fboximages.fptplay.net.vn/VOD/Documenta...


In [5]:
data_search = data_df["TitleEN"].values.tolist()
data_search

['The Amazon River Of The Sun',
 'Digging Up The Marrow',
 'Japans Tsunami - How It Happened',
 'I Am Bruce Lee',
 'The Buddha',
 'Clash Of Gods',
 'USS Ronald Reagan Airport At Sea',
 'Seasons',
 'Attack And Defend',
 'Kedi',
 "Voyage Of Time: Life's Journey",
 "The Dolomites - In The Heroes' Garden",
 'Life In The Great Wetlands',
 'Nobody, But God',
 'Mount Emei',
 'Speed Kills (Season 1)',
 'Die Kanarischen Inseln',
 'Turtle: The Incredible Journey',
 'Mysteries Of Ancient China',
 'America Wild: National Parks Adventure',
 'Journey To Space',
 'Club Friday To Be Continued: She Changed',
 'Club Friday To Be Continued Friend Against Friend',
 'Mama Does Not Like',
 'Soot Rak Chun La Moon',
 'Devil Lover',
 'Rak Long Jai',
 'Kiss The Series',
 'Xuan Yuan Sword',
 'The Eagle Shooting Heroes',
 'A Weaver On The Horizon',
 'The Vigilantes In Masks',
 'Life Of A Universe',
 'Mundo: From Altar Boy To Hitman',
 'Branded',
 'Prescient',
 'Muse',
 'Brothers In Arms',
 'Free State',
 'Flight 

In [6]:
data_search = [re.sub(r'[^a-z0-9 ]+','',i.lower()) for i in data_search]

In [7]:
data_search[:3]

['the amazon river of the sun',
 'digging up the marrow',
 'japans tsunami  how it happened']

In [8]:
string = 'the amazon river of the sun'
max_length = len(string)
space_token = string.split()
index = 0
tokens = list()
doc_tokens = dict()
for token in space_token:
    for i in range(index,index+10):
        if i+1 <= max_length:
            word = string[index:(i+1)]
            tokens.append(word)
            if word not in doc_tokens:
                doc_tokens[word] = 1
            else:
                doc_tokens[word] += 1
        else:
            break
    index += len(token) + 1
    
vocabs = set() 
    
def edge_ngram(string, n=5):
    global vocabs
    max_length = len(string)
    space_token = string.split()
    index = 0
    tokens = list()
    doc_tokens = dict()
    for token in space_token:
        for i in range(index,index+10):
            if i+1 <= max_length:
                word = string[index:(i+1)]
                tokens.append(word)
                if word not in doc_tokens:
                    doc_tokens[word] = 1
                else:
                    doc_tokens[word] += 1
            else:
                break
        index += len(token) + 1
    doc_tokens["term_counts"] = len(tokens)
    doc_tokens["doc_length"] = len(string)
    vocabs = vocabs.union(set(tokens))
    return doc_tokens

In [9]:
N = len(data_search)
term_counts = []

for doc in data_search:
    term_counts.append(edge_ngram(doc, n=10))

In [10]:
term_count_in_document = dict()
for term in vocabs:
    term_count_in_document[term] = 0
    for doc in term_counts:
        if term in doc:
            term_count_in_document[term] += 1

In [11]:
avg_length_doc = sum([i["doc_length"] for i in term_counts])/len(term_counts)
avg_length_doc

17.530647985989493

In [12]:
def tf(term, doc):
    n = doc["term_counts"]
    term_count = 0
    if term in doc:
        term_count = doc[term]
    return term_count/n

In [13]:
def tf_norm(term, doc, avg_len, k=1.2, b=0.5):
    return (tf(term, doc)*(k+1))/(tf(term, doc)+ k*(1 - b + b*doc["doc_length"]/avg_len))

In [14]:
import math
def idf(word, D):
    d = 1
    if word in D:
        d = D[word]
    return math.log2(N/d)


def tf_idf(term, doc, D):
    return tf(term, doc)*idf(term, D)

In [15]:
def idf_norm(word, D):
    d = 0
    if word in D:
        d = D[word]
    return math.log2((N-d+0.5)/(d+0.5) + 1)

In [16]:
def BM25(Q, doc):
    return sum([idf_norm(w, doc)*tf_norm(w, doc, avg_length_doc) for w in Q])    

In [17]:
result = []

for i in range(len(term_counts)):
    _1 = tf_idf('h', term_counts[i], term_count_in_document)
    _2 = tf_idf('hu', term_counts[i], term_count_in_document)
    _3 = tf_idf('hug', term_counts[i], term_count_in_document)
    _4 = tf_idf('hug ', term_counts[i], term_count_in_document)
    _5 = tf_idf('hug t', term_counts[i], term_count_in_document)
    _6 = tf_idf('hug t', term_counts[i], term_count_in_document)
    result.append(sum([_1, _2, _3, _4, _5, _6]))
    


In [18]:
result1, result2 = [], []
for i in range(len(term_counts)):
    score1 = tf_idf('trang ', term_counts[i], term_count_in_document)
    score2 = tf_idf('trang', term_counts[i], term_count_in_document)
    if score1 > 0:
        result1.append({str(i):score1})
    if score2 > 0:
        result2.append({str(i):score2})
    
top_score1 = sorted(result1,key=lambda k: list(k.values())[0], reverse=True)
top_1= top_score1[:5]
top_score2 = sorted(result2,key=lambda k: list(k.values())[0], reverse=True)
top_2= top_score2[:5]
top_score = top_1 + top_2

for res in top_score[:5]:
    print(data_search[int(list(res.keys())[0])])

In [19]:
BM25(["hug t", "hug ", "hug", "hu", "h", "t"],term_counts[271])*term_counts[271]["term_counts"]

0.0

In [20]:
BM25(["hug t", "hug ", "hug", "hu", "h", "t"],term_counts[631])*term_counts[631]["term_counts"]
# term_counts[631]

12.058236181022911

In [21]:
result3 = []
for i in range(len(term_counts)):
    score3 = BM25(["trang", "tran", "tra", "tr", "t"],term_counts[i])*term_counts[i]["term_counts"]
    if score3 > 0:
        result3.append({str(i):score3})
        
top_score3 = sorted(result3,key=lambda k: list(k.values())[0], reverse=True)        
for res in top_score3[:5]:
    print(data_search[int(list(res.keys())[0])])

trance
lost transmissions
train
bureau of transformer
the road the tragedy of one
